In [1]:
import numpy as np
import os
import sys
import argparse
import glob
import time
sys.path.append("/home/jungmo/apps/caffe/python") # caffe path

import caffe
from shutil import copyfile

#GPUID = 1
BIRD_ROOT = "/home/jungmo/Documents/bird/caffe/"

NPY_PATH = os.path.join(BIRD_ROOT,"model","new_16_200", "mean.npy")
CAFFE_BIN = "/home/jungmo/apps/caffe/build/tools/"
network = 'googLeNet'
IMG_SIZE = 224

def binaryproto_to_npy():
    blob = caffe.proto.caffe_pb2.BlobProto()
    data = open( BINARYPROTO_PATH , 'rb' ).read()
    blob.ParseFromString(data)
    arr = np.array( caffe.io.blobproto_to_array(blob) )
    out = arr[0]
    np.save( NPY_PATH, out )

def evaluate_model(dst):
        input_file = os.path.join(TEST)
        output_file = os.path.join(BIRD_ROOT, "db/output.npy")
        model_def = os.path.join(BIRD_ROOT, "model_prototxt/"+network+"/deploy.prototxt")
        pretrained_model = os.path.join(BIRD_ROOT, "model/new_16_200/googlenet_iter_60000.caffemodel")
        gpu = True
        images_dim='224,224'
        mean_file = NPY_PATH
        ext='bmp'
        channel_swap='2,1,0'
        raw_scale = 255.0
        center_only=True
        input_scale = None
        image_dims = [int(s) for s in images_dim.split(',')]

        mean, channel_swap = None, None
        if mean_file:
            mean = np.load(mean_file)
        if channel_swap:
            channel_swap=[int(s) for s in args.channel_swap.split(',')]
        caffe.set_mode_gpu()
        #caffe.set_device(GPUID)
        # Make classifier.
        classifier = caffe.Classifier(model_def, pretrained_model,
                image_dims=image_dims, mean=mean,
                input_scale=input_scale, raw_scale=raw_scale,
                channel_swap=channel_swap)

        # Load numpy array (.npy), directory glob (*.jpg), or image file.
        imgs_list = tuple(open(input_file,'r'))
        print("Loading file: %s" % input_file)
        inputs =[caffe.io.load_image(im_f)
                     for im_f in glob.glob(input_file + '/*.' + ext)]

        print("Classifying %d inputs." % len(imgs_list))

        BATCH_SIZE = 100

        # Classify.
        confusion_matrix = np.zeros((4,4),dtype=np.int32)
        accuracy = 0.0
        start = time.time()
        for i in range(0, len(imgs_list), BATCH_SIZE):
            if i > 0 and i % (BATCH_SIZE * 25) == 0:
                print("Finished %d imgs." % (i))
                cur_accuracy = accuracy / i * 100
                print("Accuracy: %.2f " % cur_accuracy)
                print("Confusion Matrix:")
                print confusion_matrix
            imgs = [imgs_list[i + j].split(' ')[0] for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            gts = [int(imgs_list[i + j].split(' ')[1]) for j in range(min(BATCH_SIZE,len(imgs_list)-i))]
            inputs = [caffe.io.load_image(path) for path in imgs]
            predictions = classifier.predict(inputs, not center_only)
            for j in range(len(imgs)):
                probs = predictions[j]
                pred = np.argmax(probs)
                confusion_matrix[gts[j]][pred] += 1
                if pred == gts[j]:
                    accuracy += 1.0

        accuracy = accuracy / len(imgs_list) * 100

        print("Done in %.2f s." % (time.time() - start))
        # Save
        print("Accuracy: %.2f " % accuracy)
        print("Confusion Matrix:")
        print confusion_matrix
        np.save(dst, confusion_matrix)

In [2]:

'''    
print "Make DB : Training set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE)+ " / " + TRAIN + " " + BIRD_ROOT + "db/train_imageData_lmDB")

print "Make DB : Test set"
os.system(os.path.join(CAFFE_BIN,"convert_imageset.bin")+" -backend=\"lmdb\" -shuffle=true -resize_height "+str(IMG_SIZE)+" -resize_width "+str(IMG_SIZE) + " / " + TEST + " " + BIRD_ROOT + "db/val_imageData_lmDB")

print "Make image mean file"
os.system(os.path.join(CAFFE_BIN,"compute_image_mean.bin")+
               " -backend=lmdb "+ BIRD_ROOT + "db/train_imageData_lmDB " + BIRD_ROOT+ "db/mean.binaryproto")
print "Image mean file .binaryproto to .npy"
binaryproto_to_npy()   

print "Training model"
os.system(os.path.join(CAFFE_BIN,"caffe.bin")+" train "
               "-solver="+BIRD_ROOT+"model/"+network+"/solver.prototxt "+" -weights="+BIRD_ROOT+"model/"+network+"/bvlc_googlenet.caffemodel "+ "-gpu all")
'''
print "Evaluate model"
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_200.txt"
evaluate_model("test_16_200.npy")


Evaluate model
Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_200.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 96.44 
Confusion Matrix:
[[2411   30   28   31]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 96.92 
Confusion Matrix:
[[4846   42   48   64]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 96.81 
Confusion Matrix:
[[6530   59   63   86]
 [   3  731    1   27]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 96.66 
Confusion Matrix:
[[8675   64   84  146]
 [   3  991    1   36]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.23 
Confusion Matrix:
[[9630   66   96  175]
 [   3  991    1   36]
 [  22    9 1158  313]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 91.30 
Confusion Matrix:
[[9630   66   96  175]
 [   3  991    1   36]
 [  66   28 3074  834]
 [   0   

In [3]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_150.txt"
evaluate_model("test_16_150.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_150.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.80 
Confusion Matrix:
[[2445   24   15   16]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.54 
Confusion Matrix:
[[4877   36   46   41]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.16 
Confusion Matrix:
[[6565   49   64   60]
 [   9  722    0   31]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.11 
Confusion Matrix:
[[8735   49   72  113]
 [  11  976    1   43]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.05 
Confusion Matrix:
[[9692   52   78  145]
 [  11  976    1   43]
 [  17    4 1088  393]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 90.83 
Confusion Matrix:
[[9692   52   78  145]
 [  11  976    1   43]
 [  55   11 2957  979]
 [   0    0    0    0]]


In [4]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_140.txt"
evaluate_model("test_16_140.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_140.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.48 
Confusion Matrix:
[[2437   20   24   19]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.62 
Confusion Matrix:
[[4881   35   45   39]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.31 
Confusion Matrix:
[[6573   45   66   54]
 [   8  725    1   28]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.09 
Confusion Matrix:
[[8732   48   79  110]
 [  11  977    1   42]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.33 
Confusion Matrix:
[[9698   48   83  138]
 [  11  977    1   42]
 [  32    5 1116  349]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 91.11 
Confusion Matrix:
[[9698   48   83  138]
 [  11  977    1   42]
 [  65    8 2991  938]
 [   0    0    0    0]]


In [ ]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_130.txt"
evaluate_model("test_16_130.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_130.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.40 
Confusion Matrix:
[[2435   18   24   23]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.72 
Confusion Matrix:
[[4886   26   50   38]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.15 
Confusion Matrix:
[[6574   41   67   56]
 [  15  712    0   35]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.15 
Confusion Matrix:
[[8744   43   78  104]
 [  17  971    1   42]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.37 
Confusion Matrix:
[[9711   44   82  130]
 [  17  971    1   42]
 [  12    4 1114  372]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 91.03 
Confusion Matrix:
[[9711   44   82  130]
 [  17  971    1   42]
 [  46   12 2973  971]
 [   0    0    0    0]]


In [11]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_120.txt"
evaluate_model("test_16_120.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_120.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.48 
Confusion Matrix:
[[2437   13   27   23]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 97.60 
Confusion Matrix:
[[4880   27   54   39]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.27 
Confusion Matrix:
[[6575   37   75   51]
 [  11  720    1   30]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.08 
Confusion Matrix:
[[8741   38   91   99]
 [  17  967    2   45]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.58 
Confusion Matrix:
[[9706   40  102  119]
 [  17  967    2   45]
 [  13    3 1149  337]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 91.70 
Confusion Matrix:
[[9706   40  102  119]
 [  17  967    2   45]
 [  32    6 3082  882]
 [   0    0    0    0]]


In [9]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_110.txt"
evaluate_model("test_16_110.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_110.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 97.80 
Confusion Matrix:
[[2445   17   17   21]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.02 
Confusion Matrix:
[[4901   23   38   38]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.44 
Confusion Matrix:
[[6600   31   59   48]
 [  17  708    2   35]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.13 
Confusion Matrix:
[[8750   34   71  114]
 [  21  963    2   45]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 94.07 
Confusion Matrix:
[[9719   34   75  139]
 [  21  963    2   45]
 [  18    1 1077  406]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 90.03 
Confusion Matrix:
[[9719   34   75  139]
 [  21  963    2   45]
 [  53    5 2823 1121]
 [   0    0    0    0]]


In [10]:
TEST = "/home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_100.txt"
evaluate_model("test_16_100.npy")

Loading file: /home/jungmo/Documents/bird/caffe/bird_no_adole/test/test_16_100.txt
Classifying 21586 inputs.
Finished 2500 imgs.
Accuracy: 98.24 
Confusion Matrix:
[[2456   10   19   15]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 5000 imgs.
Accuracy: 98.20 
Confusion Matrix:
[[4910   21   42   27]
 [   0    0    0    0]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 7500 imgs.
Accuracy: 97.63 
Confusion Matrix:
[[6623   27   47   41]
 [  31  699    1   31]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 10000 imgs.
Accuracy: 97.39 
Confusion Matrix:
[[8793   27   53   96]
 [  44  946    2   39]
 [   0    0    0    0]
 [   0    0    0    0]]
Finished 12500 imgs.
Accuracy: 93.70 
Confusion Matrix:
[[9768   30   53  116]
 [  44  946    2   39]
 [  29    1  999  473]
 [   0    0    0    0]]
Finished 15000 imgs.
Accuracy: 89.25 
Confusion Matrix:
[[9768   30   53  116]
 [  44  946    2   39]
 [  71    1 2673 1257]
 [   0    0    0    0]]
